# TREC 2018 Precision Medicine

In [5]:
import json
from json2html import *
from IPython.display import HTML
import eval
import pandas

In [6]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

## Load and split TOPICS and GOLD STANDARD from last year (30 topics)
Also, split them into training, test, and development set.

In [7]:
topics_all = eval.get_topics('./topics/topics2017.xml')
qrels_all = eval.get_qrels('./gold-standard/abstracts.2017.qrels')

In [8]:
topics_train, topics_test, topics_dev = eval.split_topics(topics_all)
qrels_train, qrels_test, qrels_dev = eval.split_qrels(qrels_all, topics_train, topics_test, topics_dev)

In [9]:
assert(set([topic for topic in topics_train['topic']])==set([topic for topic in qrels_train['topic']]))
assert(set([topic for topic in topics_test['topic']])==set([topic for topic in qrels_test['topic']]))
assert(set([topic for topic in topics_dev['topic']])==set([topic for topic in qrels_dev['topic']]))

In [10]:
full_run_df, params = eval.run(topics_all)

RUN: FIXME TOPICS: 30 {'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'disease_boost': 1.5, 'gene_boost': 1}


In [21]:
results, aggregated = eval.evaluate(qrels_all, full_run_df)
assert(aggregated['ndcg'] == 0.5338)

In [22]:
#full_run_df

Test run with default_params

In [23]:
training_run_df, training_run_params = eval.run(topics_train)

RUN: FIXME TOPICS: 12 {'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'disease_boost': 1.5, 'gene_boost': 1}


In [24]:
training_results, training_aggregated = eval.evaluate(qrels_train, training_run_df)
training_score = pandas.Series(training_aggregated, name='training ' + '(' + str(len(topics_train)) + ' topics)')

In [25]:
test_run_df, test_run_params = eval.run(topics_test, training_run_params)

RUN: FIXME TOPICS: 9 {'query_template': 'baseline.json', 'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'disease_boost': 1.5, 'gene_boost': 1}


In [26]:
training_score

P_10           0.5167
Rprec          0.3070
ndcg           0.5864
recall_1000    0.6833
Name: training (12 topics), dtype: float64

In [27]:
test_results, test_aggregated = eval.evaluate(qrels_test, test_run_df)
test_score = pandas.Series(test_aggregated, name='test ' + '(' + str(len(topics_test)) + ' topics)')

In [28]:
pandas.DataFrame([training_score, test_score])

,P_10,Rprec,ndcg,recall_1000
training (12 topics),0.5167,0.3070,0.5864,0.6833
test (9 topics),0.5333,0.2915,0.4617,0.4924


In [29]:
#eval.experiment(topics_df, qrels)